# OAI-PMH und LIDO Demo mit dem Städel Musem

Das [Städel Museum](https://www.staedelmuseum.de/) in Frankfurt am Main bietet einen freien Zugang zu seiner [Sammlung](https://sammlung.staedelmuseum.de/de). Hier können wir ebenfalls eine [OAI-PMH Schnittstelle](https://sammlung.staedelmuseum.de/de/konzept#api) für ein kleines Beispiel nutzen, um sowohl OAI-PMH, als auch das Lightweight Information Describing Objects (LIDO) vorzustellen.

Wir nutzen die [Sickl](https://sickle.readthedocs.io/) Bibliothek, welche uns ein wenig der OAI-PMH-Kommunikation versteckt. Zusätzlich habe ich in der Datei [oai_lib.py](/edit/oai_lib.py) einige weitere Helfer abgelegt.
Der Helfer `get_sickle` gibt uns eine konfigurierte Instanzt der `Sickle` Klasse zurück, welche mit der OAI-PMH-Schnittstelle der DDB verbunden ist.

> Da die späteren Zellen von vorhergehenden Zellen abhängen, sind Zellen, welche unabhängig von anderen Zellen ausgeführt werden können mit `# Einstiegspunkt` gekennzeichnet.

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle
sickle = get_sickle("https://sammlung.staedelmuseum.de/api/oai", "lido")

## Das Verb ListMetadataFormats

Damit bekommen wir eine Liste der vom Städel angebotenen Metadaten-Formate.
Wir können uns einmal die ganze Antwort der OAI-PMH-Schnittstelle anzeigen lassen … oder auch mit Hilfe der Sickl-Bibliothek durch die einzelnen Einträge iterieren.

Im Angebot sind:
- `oai_dc`: minimales Dublin Core
- `lido`: Lightweight Information Describing Objects (LIDO), *daran sind wir interessiert*

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle, display_oai, display_oai_tree
sickle = get_sickle("https://sammlung.staedelmuseum.de/api/oai", "lido")

metadata_formats = sickle.ListMetadataFormats()

display_oai_tree(metadata_formats)
display_oai(metadata_formats)

## Das Verb ListSets

Das Städel bietet uns nur zwei Sets an.

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle, display_oai, display_oai_tree
sickle = get_sickle("https://sammlung.staedelmuseum.de/api/oai", "lido")

oai_sets = sickle.ListSets()

display_oai_tree(oai_sets)
display_oai(oai_sets)

## Das Verb ListIdentifiers

Nun iterieren wir über die erhaltenen Sets in `pd_image_sets`, von welchen es jeweils `mediatype_002` (Bild) und `rights_001` (Public Domain) Varianten geben soll.

Um an die Identifier der Records zu gelangen rufen wir das Verb `ListIdentifiers` mit dem entsprechenden Identifier des Sets mit der Variante `mediatype_002` auf.
Die Liste der Identifier der Records wird nochmal filtert, da wir nur an Records interessiert sind, welche auch in `rights_001` (Public Domain) Sets sind. Für unser Beispiel reicht es das Ergebnis auf 5 Einträge zu beschränken.

Zusätzlich benötigt die Schnittselle die Angabe des Metadaten-Formats (`metadataPrefix`) und wir möchten gelöschte Einträge ignorieren.
Falls ein Set leer ist ignorieren wir dieses und gehen zum nächsten Set weiter (`try: … except NoRecordsMatch: continue`).

Die Ausgabe zeigt die Antwort der Schnittstelle auf den Aufruf mit dem Verb `ListIdentifiers`.

In [ ]:
# Einstiegspunkt

from sickle.oaiexceptions import NoRecordsMatch
from rdflib import Graph
from oai_lib import get_sickle, display_oai
sickle = get_sickle("https://sammlung.staedelmuseum.de/api/oai", "lido")

identifiers = sickle.ListIdentifiers(metadataPrefix="lido", set="scope:graphics", ignore_deleted=True)

display_oai(identifiers)

## Das Verb GetRecord

Nun haben wir eine Liste von Identifiern von Records in `found_records`.
Wir iterieren über diese Liste und rufen die Schnittstelle mit dem Verb `GetRecord` unter Angabe des Identifiers und des gewünschten Metadaten-Formats (`metadataPrefix`) auf. Die Ausgabe zeigt den zurückgegebenen Record an. Die durch die Schnittstellen übertragenen EDM-Metadaten sind innerhalb des `metadata`-Tags als XML-Baum.

In [ ]:
# Hängt von sickle und identifiers ab
from oai_lib import display_oai, display_xml, display_xml_tree

item = next(identifiers)
record = sickle.GetRecord(metadataPrefix="lido", identifier=item.identifier)
display_oai(record)
#display_xml(record.metadata)
#display_xml_tree(record.metadata, "LR")

## Beispielhafte Ausgabe der Metadaten

Es werden einige Metadaten über xpath queries aus den LIDO Daten abgefragt.

> Zwar stehen für Repository (Museum), Objektart und die Angaben zum Werkinhalt auch lesbare Bezeichner zur Verfügung, jedoch soll hier die Verknüpfung mit kontrolleirten Vokabularen demonstriert werden (GND, ICONCLASS, ISIL).


In [ ]:
# Hängt von record ab
from IPython.display import display, Markdown

namespaces = {p: n for p, n in record.metadata.nsmap.items() if p is not None}

# descriptiveMetadata
identificationWrap = record.metadata.xpath('//lido:lido/lido:descriptiveMetadata/lido:objectIdentificationWrap', namespaces=namespaces)[0]
relationWrap = record.metadata.xpath('//lido:lido/lido:descriptiveMetadata/lido:objectRelationWrap', namespaces=namespaces)[0]
eventWrap = record.metadata.xpath('//lido:lido/lido:descriptiveMetadata/lido:eventWrap', namespaces=namespaces)[0]

titel = identificationWrap.xpath('//lido:titleWrap/lido:titleSet[@lido:type="http://vocab.getty.edu/aat/300417200"]/lido:appellationValue[@xml:lang="de"]/text()', namespaces=namespaces)
iri_museum = identificationWrap.xpath('//lido:repositoryWrap/lido:repositorySet/lido:repositoryName/lido:legalBodyID/text()', namespaces=namespaces)
objekt_art = identificationWrap.xpath('//lido:objectWorkTypeWrap/lido:objectWorkType/lido:conceptID/text()', namespaces=namespaces)

werkinhalt = relationWrap.xpath('//lido:subjectWrap/lido:subjectSet/lido:subject/lido:subjectConcept/lido:conceptID[@lido:type="http://terminology.lido-schema.org/identifier_type/uri"]/text()', namespaces=namespaces)

events = eventWrap.xpath('//lido:eventSet/lido:event/lido:eventType/lido:conceptID/text()', namespaces=namespaces)
kuenstler = eventWrap.xpath('//lido:eventSet/lido:event[lido:eventType/lido:conceptID/text() = "http://terminology.lido-schema.org/eventType/production"]/lido:eventActor/lido:displayActorInRole[@xml:lang="de"]/text()', namespaces=namespaces)

# administrativeMetadata
administrativeMetadata = record.metadata.xpath('//lido:lido/lido:administrativeMetadata', namespaces=namespaces)[0]

vorschau = administrativeMetadata.xpath('//lido:resourceWrap/lido:resourceSet/lido:resourceRepresentation[@lido:type="http://terminology.lido-schema.org/lido00451"]/lido:linkResource/text()', namespaces=namespaces)
representation_lieferung = administrativeMetadata.xpath('//lido:resourceWrap/lido:resourceSet/lido:resourceRepresentation[@lido:type="http://terminology.lido-schema.org/lido00464"]/lido:linkResource/text()', namespaces=namespaces)
creditLine = administrativeMetadata.xpath('//lido:rightsWorkWrap/lido:rightsWorkSet/lido:creditLine/text()', namespaces=namespaces)

display(Markdown(f"""
| | |
|-|-|
| Titel | {titel[0]} |
| Künstler | {kuenstler[0]} |
| Repository (Museum) | {iri_museum[0]} |
| Objektart | {objekt_art[0]} |
| Werkinhalt | {werkinhalt} |
| Ereignisse | {events} |

Metadata: {creditLine[0]}

[![{titel[0]}]({vorschau[0]})]({representation_lieferung[0]})

Bild: CC BY-SA 4.0 Städel Museum, Frankfurt am Main

"""))